<h1>Section 3.2.1 A Coal Distribution Problem</h1>

In this notebook we use <code>pulp</code> to solve the coal distribution problem.  We start by loading the necessary packages and entering the data from the text as an array.

In [0]:
!pip install pulp

import numpy as np
import pulp

In [0]:
MineToCityCosts=np.array([
    [2,3,2,4],
    [4,2,2,1],
    [3,4,3,1]
])
print(MineToCityCosts)
print(MineToCityCosts[0][1])

As always with pulp, we create an object for the LP to which we will add the objective function and constraints.

In [0]:
CoalLP = pulp.LpProblem("Coal_Distribution_Problem",pulp.LpMinimize)

We will need to create 12 decision variables.  To begin we create a list of the mines and a list of the cities.  We use integers, rather than strings, in the lists so that they may also index the rows and columns of the cost matrix.

The code <code>Mines=range(3)</code> is essentially equivalent to <code>Mines=[0,1,2]</code> 

In [0]:
Mines=range(3)
Cities=range(4)

Now we create dictionary for the decision variables.  This means that instead of indivicually creating each of the 12 variables with a command such as <code>x12 = LPVariable(...)</code>, we create all of them at once.  The dictionary is named <code>x</code> and the decision variables are accessed by <code>x[1][2]</code>, for example.  (The indexing is slightly different from the text, since Python always begins with 0.  That means <code>x[0][2]</code> is a decision variable but <code>x[3][2]</code> is not.)

We include options that the variables have a minimum value of 0, no upper bound, and may take non-integer values.

In [0]:
x = pulp.LpVariable.dicts("x",(Mines,Cities),0,None,pulp.LpContinuous)

Next we include the objective function.  We multiply each decision variable, representing the amount shipped from a particular mine to a particular city, and the associated cost of shipment given by the matrix entry.  Then we sum these 12 terms to give the total cost.

In [0]:
CoalLP += sum(x[mine][city]*MineToCityCosts[mine,city] for mine in Mines for city in Cities), "Total_Cost"

In [0]:
CoalLP

Next we add our constraints.  We need constraints that we meet demand in each city, and that we do not exceed supply from any mine.

In [0]:
Demand=[4,3,5,2]

for city in Cities:
    CoalLP += sum(x[mine][city] for mine in Mines) >= Demand[city]

In [0]:
Supply=[5,5,4]

for mine in Mines:
    CoalLP += sum(x[mine][city] for city in Cities) <= Supply[mine]

In [0]:
CoalLP

In [0]:
CoalLP.solve()
print("Status:", pulp.LpStatus[CoalLP.status])

In [0]:
pulp.value(CoalLP.objective)

In [0]:
for v in CoalLP.variables():
    print(v.name, "=", v.varValue)

We might want to only print the decision variables with positive values.  The following code shows one method.

In [0]:
for v in CoalLP.variables():
    if v.varValue > 0.1:
        print(v.name, "=", v.varValue)

It is also possible to convert the solution to an array to be read more easily.  Here is one method.

In [0]:
SolutionArray=np.asarray([[x[mine][city].varValue for city in Cities] for mine in Mines])
print(SolutionArray)